In [1]:

from datasets import load_dataset

datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')



Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.


In [2]:

datasets


DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [3]:

datasets["train"]


Dataset({
    features: ['text'],
    num_rows: 36718
})

In [4]:

datasets["train"][15:18]


{'text': [' The game takes place during the Second Europan War . Gallian Army Squad 422 , also known as " The Nameless " , are a penal military unit composed of criminals , foreign deserters , and military offenders whose real names are erased from the records and thereon officially referred to by numbers . Ordered by the Gallian military to perform the most dangerous missions that the Regular Army and Militia will not do , they are nevertheless up to the task , exemplified by their motto , Altaha Abilia , meaning " Always Ready . " The three main characters are No.7 Kurt Irving , an army officer falsely accused of treason who wishes to redeem himself ; Ace No.1 Imca , a female Darcsen heavy weapons specialist who seeks revenge against the Valkyria who destroyed her home ; and No.13 Riela Marcellis , a seemingly jinxed young woman who is unknowingly a descendant of the Valkyria . Together with their fellow squad members , these three are tasked to fight against a mysterious Imperial un

In [5]:

my_own_datasets = load_dataset("text", data_files={ "train": "/home/rcalix/Desktop/rc_train.txt", 
                                                    "validation": "/home/rcalix/Desktop/rc_validation.txt"} )


In [6]:

my_own_datasets


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 4
    })
})

In [7]:

my_own_datasets["train"][:]


{'text': ['The game takes place during the Second Europan War . Gallian Army Squad 422 , also known as " The Nameless " , are a penal military unit composed of criminals , foreign deserters , and military offenders whose real names are erased from the records and thereon officially referred to by numbers . Ordered by the Gallian military to perform the most dangerous missions that the Regular Army and Militia will not do , they are nevertheless up to the task , exemplified by their motto , Altaha Abilia , meaning " Always Ready . " The three main characters are No.7 Kurt Irving , an army officer falsely accused of treason who wishes to redeem himself ; Ace No.1 Imca , a female Darcsen heavy weapons specialist who seeks revenge against the Valkyria who destroyed her home ; and No.13 Riela Marcellis , a seemingly jinxed young woman who is unknowingly a descendant of the Valkyria . Together with their fellow squad members , these three are tasked to fight against a mysterious Imperial uni

In [8]:

my_own_datasets["validation"][:]


{'text': ['The gother other s , foreign deserters , and military offenders whose real names are erased from the records and thereon officially referred to by numbers . Ordered by the Gallian military to perform the most dangerous missions that the Regular Army and Militia will not do , they are nevertheless up to the task , exemplified by their motto , Altaha Abilia , meaning " Always Ready . " The three main characters are No.7 Kurt Irving , an army officer falsely accused of treason who wishes to redeem himself ; Ace No.1 Imca , a female Darcsen heavy weapons specialist who seeks revenge against the Valkyria who destroyed her home ; and No.13 Riela Marcellis , a seemingly jinxed young woman who is unknowingly a descendant of the Valkyria . Together with their fellow squad members , these three are tasked to fight against a mysterious Imperial unit known as Calamity Raven , consisting of mostly Darcsen soldiers .',
  "As tdfdfdfdfdfd  defer wise make Gallia lose face in the war . Whil

In [9]:

from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML


In [10]:



def show_random_elements(dataset, num_examples=20):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    
    picks = []
    
    for _ in range( num_examples ):
        
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame( dataset[picks] )        ## indexing 10 picks 
    
    print(df)
    print(dataset.features.items())
    
    for column, typ in dataset.features.items():
        print(column)
        print(typ)
        print(ClassLabel)
        ## The isinstance() function returns True if the specified object 
        ## is of the specified type, otherwise False
        if isinstance(typ, ClassLabel):
            print("Hello")
            df[column] = df[column].transform(lambda i: typ.names[i])
            print(typ.names[i])
            
    display(HTML(df.to_html()))


In [11]:

show_random_elements(datasets["train"])


                                                 text
0                                   = Fern Hobbs = \n
1    The single most important prognostic factor i...
2                       = Le souper de Beaucaire = \n
3                                                    
4                                                    
5    Illinois was laid down on 10 February 1897 by...
6                                                    
7                                                    
8    The chancel contains memorials to the Lords S...
9    Fowler 's return against Birmingham City in F...
10                     = = = Hurricane Alice = = = \n
11               Harry Seawright , federal judge . \n
12   Frederick Reines was born in Paterson , New J...
13                                                   
14   During Roman Britain , what is now the Strand...
15   On 27 December 2006 , it was revealed that th...
16   On October 18 , 2010 , Dylan released Volume ...
17                          

,text
0,= Fern Hobbs = \n
1,"The single most important prognostic factor in AML is cytogenetics , or the chromosomal structure of the leukemic cell . Certain cytogenetic abnormalities are associated with very good outcomes ( for example , the ( 15 ; 17 ) translocation in acute promyelocytic leukemia ) . About half of people with AML have "" normal "" cytogenetics ; they fall into an intermediate risk group . A number of other cytogenetic abnormalities are known to associate with a poor prognosis and a high risk of relapse after treatment . \n"
2,= Le souper de Beaucaire = \n
3,
4,
5,"Illinois was laid down on 10 February 1897 by the Newport News Shipbuilding & Dry Dock Company of Newport News , Virginia . She was launched on 4 October 1898 , sponsored by Miss Nancy Leiter , daughter of Chicago multi @-@ millionaire Levi Leiter and commissioned on 16 September 1901 . The ship 's first commander was Captain George A. Converse . Illinois was the first member of her class to be authorized , but the last to enter service . After commissioning , the ship began a shakedown cruise in the Chesapeake Bay , followed by initial training . She left the area on 20 November to test a new floating dry dock in Algiers , Louisiana . The ship was back in Newport News in January 1902 . She served briefly as the flagship of Rear Admiral Robley D. Evans from 15 to 28 February ; during this period , she took part in a reception for Prince Henry of Prussia , the brother of the German Kaiser . \n"
6,
7,
8,"The chancel contains memorials to the Lords Stanley of Alderley . The memorial to John Stanley , 1st Baron Stanley of Alderley contains his effigy dressed in peer 's robes lying under a canopy with his hand on a book , dated 1856 and by Richard Westmacott . On the other side of the chancel is a memorial to his son Edward Stanley , his effigy holding a scroll in his hand and with a dog at his feet . Engraved in brass on the side of the memorial are the figures of his widow and children . Lady Stanley is seated in the middle with their four surviving sons on her right , five surviving daughters on her left and three children who had died at a young age at her knee and on her lap . A memorial tablet to John Constantine Stanley , who died in 1878 , is by Joseph Boehm . The chancel contains a monument to Rev. Edward Shipton , rector of the church from 1625 to 1630 . \n"
9,"Fowler 's return against Birmingham City in February 2006 was labelled by the tabloid press as the stuff of fairytales , and he himself said he felt like "" a kid waking up on Christmas morning every day "" . Fowler 's first appearance back at Anfield was as a substitute against Birmingham , receiving a standing ovation upon his introduction . After his return , he had three goals ruled out for offside , before finally getting off the mark on 15 March 2006 in a home game against Fulham , the same opponents against which he scored his first ever goal for Liverpool 13 years earlier . \n"


In [12]:

model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"



For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text 

 The labels will be the same as the inputs, shifted to the left.


In [13]:

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)


In [14]:

def tokenize_function(examples):
    return tokenizer(examples["text"])


In [15]:

tokenized_datasets = datasets.map(
          tokenize_function, 
          batched=True, 
          num_proc=4, 
          remove_columns=["text"]
)


Parameter 'function'=<function tokenize_function at 0x2ae993c391f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [16]:

tokenized_datasets


DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3760
    })
})

In [17]:

## try several indices 

tokenized_datasets["train"][1]


{'input_ids': [238, 8576, 9441, 2987, 238, 252],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [18]:

# block_size = tokenizer.model_max_length
block_size = 128


In [19]:

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    
    ## concatenated_examples2 = {k: examples[k] for k in examples.keys()}
    ## print(concatenated_examples2)
    ## input()
    
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    
    # Split by chunks of max_len.
    result = {
        k: [ t[i : i + block_size] for i in range(0, total_length, block_size) ]
                    for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


In [20]:

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [21]:

tokenized_datasets.keys()


dict_keys(['test', 'train', 'validation'])

In [22]:

for k in tokenized_datasets.keys():
    print( tokenized_datasets[k] ) 


Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 4358
})
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 36718
})
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3760
})


In [23]:

examples1 =  tokenized_datasets


In [24]:

## concatenated_examples3 = {k: sum(examples1[k], []) for k in examples1.keys()}
## concatenated_examples3 


In [27]:

lm_datasets["train"][4]["input_ids"]


[198,
 548,
 301,
 15410,
 555,
 227,
 198,
 1538,
 209,
 13067,
 6455,
 201,
 198,
 1538,
 2081,
 1231,
 221,
 196,
 1306,
 201,
 756,
 2726,
 684,
 293,
 7376,
 968,
 225,
 1136,
 5168,
 6395,
 209,
 24281,
 198,
 906,
 1422,
 6455,
 390,
 1136,
 277,
 2737,
 747,
 6455,
 13516,
 227,
 1579,
 5860,
 1718,
 209,
 929,
 198,
 674,
 268,
 83,
 7131,
 201,
 3079,
 3411,
 390,
 13230,
 5382,
 201,
 766,
 218,
 642,
 1763,
 196,
 4014,
 6669,
 702,
 1413,
 824,
 221,
 198,
 1231,
 218,
 198,
 674,
 209,
 1415,
 390,
 471,
 2610,
 1076,
 1388,
 2377,
 3130,
 227,
 198,
 674,
 268,
 83,
 506,
 906,
 3641,
 828,
 201,
 1360,
 543,
 1581,
 196,
 1365,
 2978,
 1496,
 209,
 252,
 261,
 674,
 268,
 83,
 2094,
 1232,
 201,
 198,
 1028,
 73,
 52,
 58,
 1232,
 201,
 301,
 3493,
 547,
 3660,
 340,
 7784,
 9345,
 9441,
 209]

In [28]:

lm_datasets["train"][4]["labels"]


[198,
 548,
 301,
 15410,
 555,
 227,
 198,
 1538,
 209,
 13067,
 6455,
 201,
 198,
 1538,
 2081,
 1231,
 221,
 196,
 1306,
 201,
 756,
 2726,
 684,
 293,
 7376,
 968,
 225,
 1136,
 5168,
 6395,
 209,
 24281,
 198,
 906,
 1422,
 6455,
 390,
 1136,
 277,
 2737,
 747,
 6455,
 13516,
 227,
 1579,
 5860,
 1718,
 209,
 929,
 198,
 674,
 268,
 83,
 7131,
 201,
 3079,
 3411,
 390,
 13230,
 5382,
 201,
 766,
 218,
 642,
 1763,
 196,
 4014,
 6669,
 702,
 1413,
 824,
 221,
 198,
 1231,
 218,
 198,
 674,
 209,
 1415,
 390,
 471,
 2610,
 1076,
 1388,
 2377,
 3130,
 227,
 198,
 674,
 268,
 83,
 506,
 906,
 3641,
 828,
 201,
 1360,
 543,
 1581,
 196,
 1365,
 2978,
 1496,
 209,
 252,
 261,
 674,
 268,
 83,
 2094,
 1232,
 201,
 198,
 1028,
 73,
 52,
 58,
 1232,
 201,
 301,
 3493,
 547,
 3660,
 340,
 7784,
 9345,
 9441,
 209]

In [29]:

tokenizer.decode(lm_datasets["train"][1]["input_ids"])


' the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Ozawa. A large'

In [30]:

tokenizer.decode(lm_datasets["train"][1]["labels"])


' the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Ozawa. A large'


First note that we duplicate the inputs tolabels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.


In [31]:

from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model  = AutoModelForCausalLM.from_config(config)


In [32]:

from transformers import Trainer, TrainingArguments


In [33]:

training_args = TrainingArguments(
    f"/scratch/scholar/rcalix/{model_checkpoint}-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False
)


In [34]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [35]:

trainer.train()


Epoch,Training Loss,Validation Loss
1,6.548100,6.475266
2,6.191400,6.202877
3,6.013000,6.115944


TrainOutput(global_step=6747, training_loss=6.389684912044242, metrics={'train_runtime': 1043.7357, 'train_samples_per_second': 51.711, 'train_steps_per_second': 6.464, 'total_flos': 3525678710784000.0, 'train_loss': 6.389684912044242, 'epoch': 3.0})

In [36]:

import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


Perplexity: 453.02



The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you would need a larger dataset and more epochs.



## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by [MASK]) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens). 

We will use the bert-base-cased model for this example. 


In [37]:

model_checkpoint     = "bert-base-cased"
tokenizer_checkpoint = "sgugger/bert-like-tokenizer"


In [38]:

tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])


Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (571 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [39]:

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [40]:

from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained(model_checkpoint)
model  = AutoModelForMaskedLM.from_config(config)


In [41]:

training_args = TrainingArguments(
    "/scratch/scholar/rcalix/test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False
    ## push_to_hub_model_id=f"{model_checkpoint}-wikitext2",
)



Finally, we use a special data_collator. 

The data_collator is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. 

Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the data_collator, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a DataCollatorForLanguageModeling. We can adjust the probability of the masking:


In [42]:

from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)


In [43]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [44]:

trainer.train()


Epoch,Training Loss,Validation Loss
1,7.100500,7.057284
2,6.889600,6.890655
3,6.872200,6.889329


TrainOutput(global_step=7038, training_loss=7.043899787627489, metrics={'train_runtime': 1211.3565, 'train_samples_per_second': 46.463, 'train_steps_per_second': 5.81, 'total_flos': 3703423157830656.0, 'train_loss': 7.043899787627489, 'epoch': 3.0})

In [45]:

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


Perplexity: 990.34



## More on the grouping function
